In [ ]:
from datetime import datetime

from nyct_gtfs import NYCTFeed

# Load the realtime feed from the MTA site
feed = NYCTFeed("N")

trains = feed.filter_trips(line_id=['N'], underway=True)
northbound_trains = [train for train in trains if train.direction == 'N']
southbound_trains = [train for train in trains if train.direction == 'S']


print(northbound_trains[0])
northbound_trains[0]

In [ ]:
import pandas as pd
df = pd.read_csv('data_files/stop_times_07_18_2025.csv')
filtered_df = df[
    df['trip_id'].str.contains(r'088950_N\.\.N33R', na=False)
]
filtered_df

In [ ]:
from nyct_gtfs.gtfs_static_types import Stations

n_stations = set(filtered_df["stop_id"])
s = Stations()
for station in n_stations:
    print(s.get_station_name(stop_id=station))

In [138]:
import datetime
import pandas as pd
from nyct_gtfs import NYCTFeed

# 1. Load static schedule
stops = pd.read_csv("data_files/stops.txt")
stop_times = pd.read_csv('data_files/stop_times_07_18_2025.csv')

# 2. Fetch real-time feed for NQRW lines
feed = NYCTFeed("N")  # feed group for N/Q/R/W
trips = feed.filter_trips(line_id=['N'], underway=True)  # list of Trip objects

results = []

# 3. Compare real-time position to schedule
for train in trips:
    trip_id = train.trip_id
    current_time = train.last_position_update
    stop_id = train.location  # e.g., "N33R"
    time_updates = train.stop_time_updates
    if stop_id and trip_id:
        # Find scheduled arrival for this trip and stop        
        sched = stop_times[
            stop_times.trip_id.str.contains(trip_id, na=False) &
            (stop_times.stop_id == stop_id)
        ][["arrival_time", "departure_time"]]
        if not sched.empty:
            scheduled_time = datetime.datetime.strptime(sched["arrival_time"].iloc[0], "%H:%M:%S").time()
            # Compare actual vs scheduled
            scheduled_dt = pd.Timestamp.combine(current_time.date(), scheduled_time)
            delay = (current_time - scheduled_dt).total_seconds() / 60.0  # in mins

            results.append({
                "trip_id": trip_id,
                "stop_id": s.get_station_name(stop_id),
                "timestamp": current_time,
                "delay_min": delay,
                "status": "on_time" if abs(delay) < 1 else "delayed" if delay > 0 else "early"
            })

# 4. Build a DataFrame and inspect
df_delays = pd.DataFrame(results)
df_delays


,trip_id,stop_id,timestamp,delay_min,status
0,094500_N..S34R,Coney Island-Stillwell Av,2025-07-18 17:01:15,1.750000,delayed
1,095050_N..N33R,30 Av,2025-07-18 17:05:57,1.950000,delayed
2,095450_N..S34R,86 St,2025-07-18 17:05:57,1.450000,delayed
3,095550_N..N33R,36 Av,2025-07-18 17:05:57,0.450000,on_time
4,096300_N..N33R,Queensboro Plaza,2025-07-18 17:05:57,-3.550000,early
5,096500_N..S34R,20 Av,2025-07-18 17:05:57,-4.050000,early
6,097100_N..S16R,Kings Hwy,2025-07-18 17:05:55,-0.083333,on_time
7,097100_N..N33R,57 St-7 Av,2025-07-18 17:05:55,-2.083333,early
8,097600_N..S34R,36 St,2025-07-18 17:05:25,-0.083333,on_time
9,098600_N..S34R,Atlantic Av-Barclays Ctr,2025-07-18 17:05:57,-2.050000,early
